# querying a local triples file using rdflib

## my use case
Examine the kind and number of statements on:
- the Bhagavad Gita written work (http://www.wikidata.org/entity/Q46802) and Annie Besant's English translation (http://www.wikidata.org/entity/Q63196925) in Wikidata
    - I can use the [WD query service](https://query.wikidata.org/) for this
- a [BF Hub/Work](https://id.loc.gov/resources/hubs/02138b5d-0a89-a6d8-1555-7f0731e6ea0c.html) (“Hub/Work”?), [BF Work](https://id.loc.gov/resources/works/1102998.html), and [BF Instance](https://id.loc.gov/resources/instances/1102998.html)
    - I need a way to query these files
    - No LOC endpoint
    - I can easily download the files


### first I need to install some libraries
I believe that I could do this in the notebook itself; but I'll use the Anaconda Prompt.
```
(base) C:\Users\Benjamin>pip install rdflib
[...stuff happens...]
(base) C:\Users\Benjamin>pip install rdflib-jsonld
[...more stuff happens...]
```
### next I need to try and figure out what I'm doing

- I'm looking at the [rdflib 6.1.1 documentation](https://rdflib.readthedocs.io/en/stable/index.html) > [Querying with SPARQL](https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html)
- Also, here's a SO post that looks my speed: [Is there a Hello World example for SPARQL with RDFLib?](https://stackoverflow.com/questions/16829351/is-there-a-hello-world-example-for-sparql-with-rdflib)

### query version one

In [ ]:
import rdflib

In [ ]:
g = rdflib.Graph()
g.parse("testdata/02138b5d-0a89-a6d8-1555-7f0731e6ea0c.rdf", format="xml")
query = "SELECT (COUNT(?property) AS ?statements) WHERE { <http://id.loc.gov/resources/hubs/02138b5d-0a89-a6d8-1555-7f0731e6ea0c> ?property ?value. }"
result = g.query(query)
for row in result:
    print(row)

In [ ]:
g2 = rdflib.Graph()
g2.parse("testdata/1102998_Work.rdf", format="xml")
query2 = "SELECT (COUNT(?property2) AS ?statements2) WHERE { <http://id.loc.gov/resources/works/1102998> ?property2 ?value2. }"
result2 = g2.query(query2)
for row in result2:
    print(row)

In [ ]:
g3 = rdflib.Graph()
g3.parse("testdata/1102998_Instance.rdf", format="xml")
query3 = "SELECT (COUNT(?property2) AS ?statements2) WHERE { <http://id.loc.gov/resources/instances/1102998> ?property2 ?value2. }"
result3 = g3.query(query3)
for row in result3:
    print(row)

#### 🔍 query version one: success with very basic queries; repeating lots of things
Things I'd like to improve:
- use one query to get statements on all three resources, rather than repeat the query for each resource/file
    - maybe JOIN?
- To do the above, would I need to load the triples from all three downloaded BF files into one graph?

**RESOURCES**
- rdflib > [Querying with SPARQL](https://rdflib.readthedocs.io/en/stable/intro_to_sparql.html#querying-with-sparql)
- a HA! [Merging graphs](https://rdflib.readthedocs.io/en/stable/merging.html#merging-graphs)

```
from rdflib import Graph
graph = Graph()
graph.parse(input1)
graph.parse(input2)
```

> graph now contains the merged graph of input1 and input2.

*It's that easy!?*

### query version two

In [ ]:
from rdflib import Graph

In [ ]:
# create a combined graph
cg = Graph()
cg.parse("testdata/02138b5d-0a89-a6d8-1555-7f0731e6ea0c.rdf", format="xml")
cg.parse("testdata/1102998_Work.rdf", format="xml")
cg.parse("testdata/1102998_Instance.rdf", format="xml")

In [ ]:
query = "SELECT (COUNT(?property) AS ?statements) WHERE { <http://id.loc.gov/resources/hubs/02138b5d-0a89-a6d8-1555-7f0731e6ea0c> ?property ?value. }"
result = cg.query(query)
for row in result:
    print(row)
query2 = "SELECT (COUNT(?property2) AS ?statements2) WHERE { <http://id.loc.gov/resources/works/1102998> ?property2 ?value2. }"
result2 = cg.query(query2)
for row in result2:
    print(row)
query3 = "SELECT (COUNT(?property2) AS ?statements2) WHERE { <http://id.loc.gov/resources/instances/1102998> ?property2 ?value2. }"
result3 = cg.query(query3)
for row in result3:
    print(row)

#### 🔍 query version two - query one graph 3x instead of three graphs 1x/each
**NOTE** that I got a different triple count for the `bf:Hub` (!?!)  
Next to do is to improve my query syntax so that one query can retrieve the statements on the Hub, the Work and the Instance

**QUESTIONS/RESOURCES/ETC.**
- I *think* that JOIN is what I need?
- How to check my SPARQL syntax before putting the query string in a var??
    - [Here](http://sparql.org/query-validator.html)'s a SPARQL query validator
    - 
- Attempting to get a query structure right in the Wikidata query service but [not having much luck so far...](https://w.wiki/57re)

### query version three

In [1]:
from rdflib import Graph
cg = Graph() #combine files
cg.parse("testdata/548884da-f5d9-240a-8ed2-c453e1183e01.rdf", format="xml")
cg.parse("testdata/9844232_Work.rdf", format="xml")
cg.parse("testdata/9844232_Instance.rdf", format="xml")

<Graph identifier=N3df76661d432471d9dcb6bbd71b61fb5 (<class 'rdflib.graph.Graph'>)>

In [3]:
# I don't think there are really 180K+ statements on the Hub, Work, and Instance...
query = "SELECT  (COUNT(( ( ?property1 + ?property2 ) + ?property3 )) AS ?statements) WHERE { <http://id.loc.gov/resources/hubs/548884da-f5d9-240a-8ed2-c453e1183e01> ?property1  ?value1 . <http://id.loc.gov/resources/works/9844232>  ?property2  ?value2 . <http://id.loc.gov/resources/instances/9844232> ?property3  ?value3 }"
result = cg.query(query)
for row in result:
    print(row)

(rdflib.term.Literal('181720', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


In [7]:
query2 = "SELECT (COUNT (?p) AS ?statements) WHERE { ?s ?p ?o . }"
result = cg.query(query2)
for row in result:
    print(row)

(rdflib.term.Literal('1302', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


In [10]:
# Really need to know more about rdflib.plugins.sparql.processor.SPARQLResult in order to print result in an understandable way...
query3 = "SELECT ?s ?p ?o WHERE { ?s ?p ?o .} LIMIT 10"
result = cg.query(query3)
print(type(result))
for row in result:
    print(row)

<class 'rdflib.plugins.sparql.processor.SPARQLResult'>
(rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85108438'), rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#authoritativeLabel'), rdflib.term.Literal('Psychological fiction.'))
(rdflib.term.BNode('Nbfc91fb33df04d26a297464efffdc8b9'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://id.loc.gov/ontologies/bibframe/Title'))
(rdflib.term.URIRef('http://id.loc.gov/resources/works/8272098'), rdflib.term.URIRef('http://id.loc.gov/ontologies/bibframe/title'), rdflib.term.BNode('Nbb121a01da3341a2ba9fffdf0ef90758'))
(rdflib.term.URIRef('http://id.loc.gov/resources/works/2249719'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://id.loc.gov/ontologies/bibframe/Work'))
(rdflib.term.URIRef('http://id.loc.gov/resources/hubs/548884da-f5d9-240a-8ed2-c453e1183e01'), rdflib.term.URIRef('http://id.loc.gov/ontologies/bflc/subjectOf'), rdf

#### 🔍 query version three - doesn't work yet
Something is going wrong in my attept to retrieve statements on 3 IRIs...

**MORE NOTES/QUESTIONS/RESOURCES**
- Need to understand [class rdflib.plugins.sparql.processor.SPARQLResult](https://rdflib.readthedocs.io/en/stable/apidocs/rdflib.plugins.sparql.html#rdflib.plugins.sparql.processor.SPARQLResult) better for printing query results? Although the docs li
